# 4 Monte Carlo Method

The Monte Carlo Method, specifically the Metropolis algorithm, is one of the most widely used tools for studying thermodynamic quantities.
It samples phase space using a random walk such that a large number of samples $X_i$ can be used to evaluate arbitrary thermodynamic expectation values $\langle f(x)\rangle$, through

$$ \langle f(x) \rangle = \frac{1}{N_s}\sum_{i=1}^{N_s} f(X_i) $$

In [ ]:
from contextlib import contextmanager
from functools import partial
from typing import Any, Callable, NamedTuple

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from spirit import parameters, quantities, simulation, state, system
from spirit.scalar import scalar


## The Metropolis algorithm using *Spirit*

Runing a Monte Carlo simulation using the metropolis algorithm requires five basic steps:
1. initialization
2. start
3. windup
4. data gathering
5. tear down

The context manager below does just that while leaving room for the data gathering step that will differ most between Monte Carlo simulations.

In the initialization step we set the spin orientations in a desired starting configuration.
This can speed up the windup phase dramatically if we start close to an energy minimum, but it is essential for the second half of this tutorial because the constrained Monte Calro algorithm can only retain the starting magnetization of the system.
We also have to set the temperature of the system as well as the parameters of the trial distribution.
In this case the simulation is set up to use an adaptive cone method with a starting angle of 80° and a target acceptance ratio of 50%.
When starting the simulation we choose the Metropolis algorithm by passing `method_type=simulation.METHOD_MC` to the `simulation.start` method and set the desired version of the algorithm that is used.
Currently there are two algorithms that are supported: the standard unconstrained Metropolis algorithm `simulation.MC_ALGORITHM_METROPOLIS` and a direction constrained version `simulation.MC_ALGORITHM_METROPOLIS_MDC`  based on [10.1103/PhysRevB.82.054415](https://doi.org/10.1103/PhysRevB.82.054415).
We also want to make sure to use a single shot simulation where simulation steps have to be run manually using the `simulation.n_shot()` method.
This also requires us to stop the simulation by calling `simulation.stop()` after we gathered all of the necessary data.
The windup ensures that we are approching a stationary distribution of the Metropolis procedure and is simply achieved by running large enough number of simulation steps using the the `simulation.n_shot()` method.

In [ ]:
@contextmanager
def mc_context(
    p_state,
    temperature,
    orientation=(0.0, 0.0, 1.0),
    n_windup=10000,
    algorithm=simulation.MC_ALGORITHM_METROPOLIS,
):
    """context manager to handle setup, windup and tear down of a MC simulation"""

    # initialization
    orientation = np.asarray(orientation, dtype=scalar)
    orientation /= np.linalg.norm(orientation)

    spin = system.get_spin_directions(p_state)
    spin[:] = orientation

    parameters.mc.set_temperature(p_state, temperature)
    parameters.mc.set_metropolis_cone(
        p_state,
        use_cone=True,
        cone_angle=80.0,
        use_adaptive_cone=True,
        target_acceptance_ratio=0.5,
    )

    # start
    simulation.start(
        p_state,
        method_type=simulation.METHOD_MC,
        solver_type=algorithm,
        single_shot=True,
    )

    # windup
    simulation.n_shot(p_state, n_windup)

    try:
        # data gathering in `with` block
        yield
    finally:
        # tear down
        simulation.stop(p_state)


## 4.1 Curie Temperature

For the atomistic Heisenberg Hamiltonian
$$ \mathcal{H} = -\sum_{\langle ij\rangle} J_{ij} \vec{n}_i \cdot \vec{n}_j $$
with $J_{ij} > 0$ the Curie temperature represents the most basic feature of the ferromagnetic phase transition.
It separates the ordered ferromagnetic phase from unordered paramagnetic phase and is characterized by the point where the absolute value of the magnetization vanishes.
For finite systems this cannot happen and the speed at which the magnetization approaches zero beyond the critical point increases with the system size.

In most practical calculations the Binder cumulant
$$ U_4 = 1 - \frac{\langle m^4 \rangle}{3\langle m^2 \rangle^2} $$
is used, because it has a fixed point that is independent of the system size.
But to keep computation time more managable for the scope of this tutorial we limit ourselves to studying the magnetiztion directly for a system of fixed size:

$$\langle m \rangle = \left\langle \left\vert \frac{1}{N} \sum_{i=1}^{N} \mu_i \vec{n}_i\right\vert \right\rangle $$

The input file uses a simple cubic lattice with nearest neighbour interactions using $J=10\mathrm{meV}$.

<b style="color: red">Task:</b> Calculate the average Magnetization by adding to the code below the `$$$` mark.<br/>
*Hint:* Have a look at the [`quantities`](https://spirit-docs.readthedocs.io/en/latest/core/docs/python-api/spirit.quantities.html) module.

In [ ]:
def mc_magnetization(config_file, temperatures, n_samples=10000):
    """simulation routine for the average magnetization"""
    with state.State(config_file, quiet=True) as p_state:
        magnetizations = []
        for temp in temperatures:
            with mc_context(
                p_state,
                temp,
                algorithm=simulation.MC_ALGORITHM_METROPOLIS,
            ):
                average_magnetization = 0.0
                for _ in range(n_samples):
                    simulation.n_shot(p_state, 1)
                    system.update_magnetization(p_state)

                    # your code here: $$$

                average_magnetization /= n_samples
            magnetizations.append(average_magnetization)
    return np.asarray(temperatures), np.asarray(magnetizations)


temperatures, magnetization = mc_magnetization(
    config_file="./input/mc_input.cfg",
    temperatures=[0.01, *range(20, 420, 20)],
    n_samples=10000,
)


In [ ]:
def plot_magnetization(temperatures, magnetization, model=None, params=None):
    """plot routine for ferromagnetic phase transition"""
    fig, ax = plt.subplots(1, 1)
    ax.scatter(temperatures, magnetization, color="k")

    ax.set_title("Magnetization")
    ax.set_ylabel(r"$\langle{m}\rangle\;[\mu_B]$")
    ax.set_xlabel("$T$ [K]")

    if model is not None and params is not None:
        xrange = np.min(temperatures), np.max(temperatures)
        x = np.linspace(xrange[0], xrange[1], 100)
        y = model(x, *params)
        ax.plot(x, y, linestyle="--", color="grey", zorder=0.5)
        label = "\n".join(
            (
                r"$T_c = 232\text{K} = \frac{z\cdot J}{3k_B}$ (mean field)",
                rf"$T_c = {params[0]:.0f}\text{{K}}$ (fit)",
                r"$T_c = 176\text{K} = 1.44\cdot\frac{J}{k_B}$ (high $T$ expansion)",
            )
        )
        bbox = {"boxstyle": "round", "facecolor": "white", "alpha": 0.8}
        alignment = {
            "transform": ax.transAxes,
            "verticalalignment": "top",
            "horizontalalignment": "right",
            "multialignment": "left"
        }
        ax.text(0.95, 0.95, label, fontsize=10, bbox=bbox, **alignment)
    elif model is None and params is None:
        ax.axhline(0.0, 1.0, 0.0, linestyle="--", color="grey", zorder=0.5)
    else:
        msg = "WARNING: cannot plot model: missing either model or model parameters"
        print(f"\x1b[33m{msg}\x1b[0m")
    return fig, ax


_ = plot_magnetization(temperatures, magnetization)


For a second-order phase transition we generally expect $\langle m\rangle(T)$ to scale as:
$$ \frac{\langle m\rangle(T)}{m_s} = \left(1 - \frac{T}{T_c}\right) ^ b $$

<b style="color: red">Task:</b> Fit such a model to the magnetization data. Add your code in place of the `$$$` mark.

In [ ]:
def magnetization_model(T, Tc, b=0.33, Ms=2.0):
    """model for second-order phase transition"""
    T = np.atleast_1d(T)
    result = np.zeros_like(T)
    # your code here: $$$
    return result


params, _ = curve_fit(magnetization_model, temperatures, magnetization, p0=(167.0, 0.33))
_ = plot_magnetization(temperatures, magnetization, magnetization_model, params)


## 4.2 Direction Constrained Monte Carlo Method

### Temperature scaling of the Magnetic Anisotropy Energy
The unconstrained Monte Carlo Method is not suited to study direction dependent phenomena.
While it is possible to bias the direction of spins by applying a magnetic field the definition of direction dependent quantities is usually defined for vanishing magnetic field.
Following [10.1103/PhysRevB.82.054415](https://doi.org/10.1103/PhysRevB.82.054415) *Spirit* implements a direction constrained Monte Carlo Method that can be used to study these quantitites.

The most fundamental direction dependent quantitiy is certainly the Magnetic Anisotropy Energy (MAE). In the case of a uniaxial anisotropy it can be written as:
$$ f(\vec{M}(T), T) = - K(T) [\hat{K}\cdot\vec{M}(T)]^2 $$
The assotiated atomistic Hamiltonian is:
$$ \mathcal{H} = -\sum_{\langle ij\rangle} J_{ij} \vec{n}_i \cdot \vec{n}_j - K \sum_i (\hat{K} \cdot \vec{n}_i)^2 $$



Generally one has to evaluate a derivative of the free energy over a curve in order to evaluate a difference of the Free energy of the two endpoints:
$$ \Delta f = \int_\gamma \frac{\partial f}{\partial\lambda} d\lambda $$

Using the angle between the fixed magnetization direction and the anisotropy axis as a parameter for a path the associated derivative of the free energy is the mean torque within the system:
$$ \frac{\partial f}{\partial\theta} = \left\langle \frac{1}{N}\sum_{i=0}^N \vec{n}_i \times \nabla_{\hat{n_i}}{\mathcal{H}} \right\rangle \cdot \frac{\partial \hat{m}}{\partial\theta} $$

Using that parametrization to evaluate the free energy we expect $\partial f / \partial\theta \propto \sin(2\theta)$:

![Torque plot](./assets/path_torque.png)

Putting that assumption into the model we can find a good estimate for the anisotropy energy constant $K$ more efficiently using:
$$ K(T) \propto \frac{\partial f}{\partial \theta}\Big(\theta=\frac{\pi}{4}\Big) - \frac{\partial f}{\partial \theta}\Big(\theta=0\Big) $$

<b style="color: red">Task:</b> Calculate the average Magnetization and the average torque from the spin directions and the effective field.<br/>
*Hint:* The [`system`](https://spirit-docs.readthedocs.io/en/latest/core/docs/python-api/spirit.system.html) module might be helpful.

In [ ]:
def direction(theta, phi):
    return np.array(
        (
            np.sin(theta) * np.cos(phi),
            np.sin(theta) * np.sin(phi),
            np.cos(theta),
        ),
        dtype=scalar,
    )


class TorqueSample(NamedTuple):
    magnetization: float
    torque: float


def mc_anisotropy(config_file, temperatures, n_samples=10000):
    """simulation routine for the relative anisotropy energy"""
    angles = (0, np.pi / 4)
    with state.State(config_file, quiet=True) as p_state:
        torque = []
        for temp in temperatures:
            angle_resolved = []
            for theta in angles:
                orientation = direction(theta=theta, phi=0.0)
                orientation = np.asarray(orientation, dtype=scalar)
                with mc_context(
                    p_state,
                    temp,
                    orientation=orientation,
                    algorithm=simulation.MC_ALGORITHM_METROPOLIS_MDC,
                ):
                    average_magnetization = 0.0
                    average_torque = 0.0
                    for _ in range(n_samples):
                        simulation.n_shot(p_state, 1)
                        system.update_magnetization(p_state)
                        system.update_effective_field(p_state)

                        # your code here: $$$

                angle_resolved.append(
                    TorqueSample(
                        magnetization=average_magnetization / n_samples,
                        torque=average_torque / n_samples,
                    )
                )
            torque.append(
                TorqueSample(
                    magnetization=0.5
                    * (
                        angle_resolved[0].magnetization
                        + angle_resolved[1].magnetization
                    ),
                    torque=angle_resolved[0].torque - angle_resolved[1].torque,
                )
            )
    (magnetization, anisotropy_energy) = zip(*torque)

    return (
        np.asarray(temperatures),
        np.asarray(magnetization),
        np.asarray(anisotropy_energy),
    )


(_, anisotropy_magnetization, anisotropy_energy) = mc_anisotropy(
    config_file="./input/mc_input-anisotropy.cfg",
    temperatures=[0.01, *range(10, 110, 10)],
    n_samples=10000,
)


Looking at the the magnetic anisotropy coefficient $K$ as a function of the magnetization this method can reproduce the famous Callen and Callen scaling law.
$$ \frac{K(T)}{K(0)} = \left(\frac{m(T)}{m(0)}\right)^3 $$
And as Asselin et. al. have demonstrated also other scaling laws when using more detailed Hamiltonians.

In [ ]:
def plot_scaling(magnetization, anisotropy_energy):
    """plot routine for the magnetization scaling of the magnetic anisotropy energy"""
    magnetization = np.asarray(magnetization)
    anisotropy_energy = np.asarray(anisotropy_energy)
    assert magnetization.shape == anisotropy_energy.shape

    xrange = (np.min(magnetization), np.max(magnetization))
    yrange = (np.min(anisotropy_energy), np.max(anisotropy_energy))

    fig, ax = plt.subplots(1, 1)
    if xrange[1] > xrange[0] or yrange[1] > yrange[0]:
        x = np.linspace(np.log10(xrange[0] / xrange[1]), 0.0, 100, endpoint=True)
        ax.plot(x, 3 * x, "k--", zorder=0.5, label="$K(T)/K(0) = (M/M_s)^3$")
        ax.scatter(
            np.log10(magnetization / xrange[1]),
            np.log10(anisotropy_energy / yrange[1]),
            label="energy ratio",
        )

    ax.set_title("Magnetic Anisotropy Energy")
    ax.set_xlabel(r"$\log( \langle{m(T)}\rangle/\langle{m(0)}\rangle )$")
    ax.set_ylabel(r"$\log( K(T)/K(0))$")
    ax.legend()

    return fig, ax


_ = plot_scaling(anisotropy_magnetization, anisotropy_energy)
